In [1]:
from pyspark import SparkContext
import datetime
from csv import reader
import pandas            as pd
import numpy             as np
import matplotlib
import matplotlib.pyplot as plt
import sys #only needed to determine Python version number
%matplotlib inline
matplotlib.style.use('ggplot')

### Data Clean

In [2]:
sc = SparkContext()
# use dataclean.py from here to clean data
filepath    = r'./NYPD_Complaint_Data_Historic.csv'
df = sc.textFile(filepath).map(lambda l: reader([l]).__next__())

header = df.first() # csv header

df = df.filter(lambda line: line != header).cache() # filter out header

In [ ]:
def transform(array):
    array[1] = datetime.datetime.strptime(array[1], '%m/%d/%Y')
    array[2] = datetime.datetime.strptime(array[2], '%H:%M:%S').time()
    return array

#replace year 1015
def replaceYear2015to2015(array):
    array[1] = array[1].replace('1015', '2015')
    return array

def replaceTime24to00(array):
    array[2] = array[2].replace('24:00:00', '00:00:00')
    return array

t = df \
    .filter(lambda array: array[1] != '' and array[2] != '') \
    .map(replaceYear2015to2015) \
    .map(replaceTime24to00) \
    .map(transform) \
    .filter(lambda array: (array[1].year >= 2006 and array[1].year <= 2015))

### Year with Crime Data

In [ ]:
def transformYearAndCoordinate(x):
    x[21] = float(x[21])
    x[22] = float(x[22])
    return (str(x[1].year), '[' + str(x[21]) + ','+ str(x[22]) + ']')

yearWithCoordinate = t \
    .filter(lambda x: x[21] != '' and x[22] != '') \
    .map(transformYearAndCoordinate)

for i in range(2006, 2016):
    yearWithCoordinate \
    .filter(lambda x: x[0] == str(i)) \
    .saveAsTextFile('year' + str(i))
            

### Month with Crime Data

In [ ]:
def transformMonthAndCoordinate(x):
    x[21] = float(x[21])
    x[22] = float(x[22])
    return (str(x[1].month), '[' + str(x[21]) + ','+ str(x[22]) + ']')

monthWithCoordinate = t \
    .filter(lambda x: x[21] != '' and x[22] != '') \
    .map(transformMonthAndCoordinate)

for i in range(1, 13):
    monthWithCoordinate \
    .filter(lambda x: x[0] == str(i)) \
    .saveAsTextFile('month' + str(i))

### Time with Crime Data

In [ ]:
def timeMap(x):
    if x[2].hour >= 0 and x[2].hour < 6:
        return ('1', '[' + str[21] + ',' + str[22] + ']')
    elif x[2].hour >= 6 and x[2].hour < 12:
        return ('2', '[' + str[21] + ',' + str[22] + ']')
    elif x[2].hour >= 12 and x[2].hour < 18:
        return ('3', '[' + str[21] + ',' + str[22] + ']')
    else:
        return ('4', '[' + str[21] + ',' + str[22] + ']')

timeWithCoordinate = t \
    .filter(lambda x: x[21] != '' and x[22] != '') \
    .map(transformTimeAndCoordinate)

for i in range(1, 5):
    timeWithCoordinate \
    .filter(lambda x: x[0] == str(i)) \
    .saveAsTextFile('time' + str(i))

### 